# Start

In [1]:
import importlib
import os
import pandas as pd
import json
from pathlib import Path
from io import StringIO

from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

In [2]:
from bsm_multi_agents.agents import utils
importlib.reload(utils)
from bsm_multi_agents.agents.utils import get_tool_result_from_messages,print_resp

In [3]:
project_path = '/Users/yifanli/Github/model_doc_automation'

# Graph

In [7]:
from langgraph.graph import StateGraph, END, START

from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.agents.data_loader_agent import data_loader_node

graph = StateGraph(WorkflowState)
graph.add_node("data_loader", data_loader_node)

graph.add_edge(START, "data_loader")
graph.add_edge("data_loader", END)

app = graph.compile()

In [8]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
init_state: WorkflowState = {
    "csv_file_path": str(file_path),
    "messages": [HumanMessage(content=f"Load CSV from: {file_path}")],
}

final_state = app.invoke(
    init_state,
    config={"configurable": {"thread_id": "run-1"}}
)

[data_loader] LLM: ChatOllama (qwen2.5:7b), Tools: ['csv_loader']


# Data Loader

## Function Level

In [5]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
df = pd.read_csv(file_path)
json_str = df.to_json(orient='records')
result = {"csv_data": json_str}

In [6]:
json.dumps(result)

'{"csv_data": "[{\\"date\\":\\"2025-09-01\\",\\"S\\":100,\\"K\\":105,\\"T\\":1.0,\\"r\\":0.05,\\"sigma\\":0.2,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-02\\",\\"S\\":102,\\"K\\":106,\\"T\\":0.9,\\"r\\":0.045,\\"sigma\\":0.19,\\"option_type\\":\\"put\\"},{\\"date\\":\\"2025-09-03\\",\\"S\\":98,\\"K\\":104,\\"T\\":0.8,\\"r\\":0.048,\\"sigma\\":0.21,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-04\\",\\"S\\":101,\\"K\\":107,\\"T\\":0.7,\\"r\\":0.047,\\"sigma\\":0.18,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-05\\",\\"S\\":99,\\"K\\":103,\\"T\\":0.6,\\"r\\":0.046,\\"sigma\\":0.22,\\"option_type\\":\\"put\\"},{\\"date\\":\\"2025-09-06\\",\\"S\\":103,\\"K\\":108,\\"T\\":0.5,\\"r\\":0.049,\\"sigma\\":0.2,\\"option_type\\":\\"put\\"},{\\"date\\":\\"2025-09-07\\",\\"S\\":97,\\"K\\":102,\\"T\\":0.4,\\"r\\":0.044,\\"sigma\\":0.23,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-08\\",\\"S\\":100,\\"K\\":106,\\"T\\":0.3,\\"r\\":0.05,\\"sigma\\":0.19,\\"option_type\\

## Tool Level

In [8]:
from bsm_multi_agents.tools import data_loader_tools
importlib.reload(data_loader_tools)
from bsm_multi_agents.tools.data_loader_tools import csv_loader

In [9]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
csv_loader.invoke(file_path)

'{"csv_data": "[{\\"date\\":\\"2025-09-01\\",\\"S\\":100,\\"K\\":105,\\"T\\":1.0,\\"r\\":0.05,\\"sigma\\":0.2,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-02\\",\\"S\\":102,\\"K\\":106,\\"T\\":0.9,\\"r\\":0.045,\\"sigma\\":0.19,\\"option_type\\":\\"put\\"},{\\"date\\":\\"2025-09-03\\",\\"S\\":98,\\"K\\":104,\\"T\\":0.8,\\"r\\":0.048,\\"sigma\\":0.21,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-04\\",\\"S\\":101,\\"K\\":107,\\"T\\":0.7,\\"r\\":0.047,\\"sigma\\":0.18,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-05\\",\\"S\\":99,\\"K\\":103,\\"T\\":0.6,\\"r\\":0.046,\\"sigma\\":0.22,\\"option_type\\":\\"put\\"},{\\"date\\":\\"2025-09-06\\",\\"S\\":103,\\"K\\":108,\\"T\\":0.5,\\"r\\":0.049,\\"sigma\\":0.2,\\"option_type\\":\\"put\\"},{\\"date\\":\\"2025-09-07\\",\\"S\\":97,\\"K\\":102,\\"T\\":0.4,\\"r\\":0.044,\\"sigma\\":0.23,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-08\\",\\"S\\":100,\\"K\\":106,\\"T\\":0.3,\\"r\\":0.05,\\"sigma\\":0.19,\\"option_type\\

## Agent Level

In [10]:
from bsm_multi_agents.agents.agent_factory import built_graph_agent_by_role
from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.prompts.loader import load_prompt

In [11]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
state = {
    "csv_file_path": file_path
}
prompt_path = os.path.join(project_path, "src/bsm_multi_agents/prompts/data_loader_prompts.txt")

In [12]:
agent_role = 'data_loader'
agent = built_graph_agent_by_role(agent_role)

[data_loader] LLM: ChatOllama (qwen2.5:7b), Tools: ['csv_loader']


In [13]:
csv_path = state.get("csv_file_path")
prompt = load_prompt(prompt_path).format(csv_path=str(csv_path))
msg = HumanMessage(content=prompt)

result = agent.invoke(
        {"messages": [msg]},
        config={
            "recursion_limit": 10,
            "configurable": {"thread_id": "run-1"}
        }
    )

In [14]:
csv_data = get_tool_result_from_messages(result["messages"], "csv_loader")

In [19]:
json.loads(csv_data["result"])['csv_data']

'[{"date":"2025-09-01","S":100,"K":105,"T":1.0,"r":0.05,"sigma":0.2,"option_type":"call"},{"date":"2025-09-02","S":102,"K":106,"T":0.9,"r":0.045,"sigma":0.19,"option_type":"put"},{"date":"2025-09-03","S":98,"K":104,"T":0.8,"r":0.048,"sigma":0.21,"option_type":"call"},{"date":"2025-09-04","S":101,"K":107,"T":0.7,"r":0.047,"sigma":0.18,"option_type":"call"},{"date":"2025-09-05","S":99,"K":103,"T":0.6,"r":0.046,"sigma":0.22,"option_type":"put"},{"date":"2025-09-06","S":103,"K":108,"T":0.5,"r":0.049,"sigma":0.2,"option_type":"put"},{"date":"2025-09-07","S":97,"K":102,"T":0.4,"r":0.044,"sigma":0.23,"option_type":"call"},{"date":"2025-09-08","S":100,"K":106,"T":0.3,"r":0.05,"sigma":0.19,"option_type":"call"},{"date":"2025-09-09","S":104,"K":109,"T":0.2,"r":0.045,"sigma":0.21,"option_type":"put"},{"date":"2025-09-10","S":96,"K":101,"T":0.1,"r":0.048,"sigma":0.2,"option_type":"put"}]'

### main level

In [20]:
from bsm_multi_agents.agents.data_loader_agent import data_loader_node

In [21]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
state = WorkflowState(csv_file_path=file_path)
out = data_loader_node(state)
out

[data_loader] LLM: ChatOllama (qwen2.5:7b), Tools: ['csv_loader']


{'messages': [HumanMessage(content='Load the option data from the CSV file at: /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv\n\nUse the csv_loader tool to read the CSV file. Return the data in JSON format.', additional_kwargs={}, response_metadata={}, id='128f3961-32d8-4135-8523-48a3dc1c1876'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-11-25T17:27:42.33347Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1184851500, 'load_duration': 488068208, 'prompt_eval_count': 256, 'prompt_eval_duration': 290731958, 'eval_count': 36, 'eval_duration': 394107918, 'model_name': 'qwen2.5:7b', 'model_provider': 'ollama'}, id='lc_run--a2ed0209-0285-4626-a494-3a6b43bcc6fa-0', tool_calls=[{'name': 'csv_loader', 'args': {'filepath': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv'}, 'id': '404cec96-c9d0-468c-a726-922ab9607296', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2

# Calculator

In [22]:
from bsm_multi_agents.agents.data_loader_agent import data_loader_node

In [23]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
state = WorkflowState(csv_file_path=file_path)
out = data_loader_node(state)
csv_data = out['csv_data']

[data_loader] LLM: ChatOllama (qwen2.5:7b), Tools: ['csv_loader']


## Function Level

In [45]:
from bsm_multi_agents.tools import bsm_calculator_tools
importlib.reload(bsm_calculator_tools)
from bsm_multi_agents.tools.bsm_calculator_tools import _bsm_calculator, _greeks_calculator
from bsm_multi_agents.tools.utils import load_json_as_df

### batch_bsm_calculator

In [27]:
df = load_json_as_df(csv_data)
df.head()

,date,S,K,T,r,sigma,option_type
0,2025-09-01,100,105,1.0,0.050,0.20,call
1,2025-09-02,102,106,0.9,0.045,0.19,put
2,2025-09-03,98,104,0.8,0.048,0.21,call
3,2025-09-04,101,107,0.7,0.047,0.18,call
4,2025-09-05,99,103,0.6,0.046,0.22,put


In [28]:
required_cols = ['option_type', 'S', 'K', 'T', 'r', 'sigma']
missing_cols = [col for col in required_cols if col not in df.columns]
missing_cols


[]

In [34]:
def calc_row(row):
    return _bsm_calculator(
        row['option_type'], 
        row['S'], 
        row['K'], 
        row['T'], 
        row['r'], 
        row['sigma']
    )
df['BSM_Price'] = df.apply(
    calc_row, axis=1
)
result = {"bsm_results": df.to_json(orient='records', date_format='iso')}
json.dumps(result)

'{"bsm_results": "[{\\"date\\":\\"2025-09-01T00:00:00.000\\",\\"S\\":100,\\"K\\":105,\\"T\\":1.0,\\"r\\":0.05,\\"sigma\\":0.2,\\"option_type\\":\\"call\\",\\"BSM_Price\\":8.0213522351},{\\"date\\":\\"2025-09-02T00:00:00.000\\",\\"S\\":102,\\"K\\":106,\\"T\\":0.9,\\"r\\":0.045,\\"sigma\\":0.19,\\"option_type\\":\\"put\\",\\"BSM_Price\\":7.2142383713},{\\"date\\":\\"2025-09-03T00:00:00.000\\",\\"S\\":98,\\"K\\":104,\\"T\\":0.8,\\"r\\":0.048,\\"sigma\\":0.21,\\"option_type\\":\\"call\\",\\"BSM_Price\\":6.415749359},{\\"date\\":\\"2025-09-04T00:00:00.000\\",\\"S\\":101,\\"K\\":107,\\"T\\":0.7,\\"r\\":0.047,\\"sigma\\":0.18,\\"option_type\\":\\"call\\",\\"BSM_Price\\":4.9529618176},{\\"date\\":\\"2025-09-05T00:00:00.000\\",\\"S\\":99,\\"K\\":103,\\"T\\":0.6,\\"r\\":0.046,\\"sigma\\":0.22,\\"option_type\\":\\"put\\",\\"BSM_Price\\":7.3776960425},{\\"date\\":\\"2025-09-06T00:00:00.000\\",\\"S\\":103,\\"K\\":108,\\"T\\":0.5,\\"r\\":0.049,\\"sigma\\":0.2,\\"option_type\\":\\"put\\",\\"BSM_Price

### batch_greeks_calculator

In [53]:
df = load_json_as_df(csv_data)
required_cols = ['option_type', 'S', 'K', 'T', 'r', 'sigma']
missing_cols = [col for col in required_cols if col not in df.columns]
missing_cols

[]

In [54]:
def calc_row(row):
    res = _greeks_calculator(
        row['option_type'], 
        row['S'], 
        row['K'], 
        row['T'], 
        row['r'], 
        row['sigma'], 
    )
    return res
expanded = df.apply(calc_row, axis=1).apply(pd.Series)
result_cols = ['price','delta','gamma','vega','rho','theta']
for col in result_cols:
    if col not in expanded:
        expanded[col] = pd.NA
df = pd.concat([df, expanded[result_cols]], axis=1)

In [56]:
result = {"greeks_results": df.to_json(orient='records', date_format='iso')}
json.dumps(result)

'{"greeks_results": "[{\\"date\\":\\"2025-09-01T00:00:00.000\\",\\"S\\":100,\\"K\\":105,\\"T\\":1.0,\\"r\\":0.05,\\"sigma\\":0.2,\\"option_type\\":\\"call\\",\\"price\\":8.0213522351,\\"delta\\":0.5422283336,\\"gamma\\":0.0198352619,\\"vega\\":39.6705238084,\\"rho\\":46.2014811233,\\"theta\\":-6.277126437},{\\"date\\":\\"2025-09-02T00:00:00.000\\",\\"S\\":102,\\"K\\":106,\\"T\\":0.9,\\"r\\":0.045,\\"sigma\\":0.19,\\"option_type\\":\\"put\\",\\"price\\":7.2142383713,\\"delta\\":-0.4596134147,\\"gamma\\":0.0215874826,\\"vega\\":38.4059448764,\\"rho\\":-48.685326005,\\"theta\\":-1.6196945478},{\\"date\\":\\"2025-09-03T00:00:00.000\\",\\"S\\":98,\\"K\\":104,\\"T\\":0.8,\\"r\\":0.048,\\"sigma\\":0.21,\\"option_type\\":\\"call\\",\\"price\\":6.415749359,\\"delta\\":0.4928141481,\\"gamma\\":0.0216695176,\\"vega\\":34.963159853,\\"rho\\":33.5040297211,\\"theta\\":-6.599156514},{\\"date\\":\\"2025-09-04T00:00:00.000\\",\\"S\\":101,\\"K\\":107,\\"T\\":0.7,\\"r\\":0.047,\\"sigma\\":0.18,\\"option